#Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/competitions/fake-news/overview

###Loading Data From Kaggle

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
!kaggle competitions download -c fake-news

 80% 37.0M/46.5M [00:00<00:00, 177MB/s] 
100% 46.5M/46.5M [00:00<00:00, 180MB/s]


In [3]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


###Importing Data 

In [4]:
import pandas as pd
data=pd.read_csv('train.csv')
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


###Removing NaN Values from the dataset

In [5]:
data=data.dropna()
data.reset_index(inplace=True)
data.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


###Extracting Dependent and Independent variables from data

In [6]:
X = data.drop('label',axis=1)
y = data['label']

###Checking The Shape of X and y

In [7]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


###Importing the required Libraries

In [8]:
import tensorboard as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

###Preprocessing The Text 



In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # stem the words that are not in stopwords
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [11]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 


###Applying Pad Sequences To Make Sentence Length Equal

In [12]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  290 2602 2358]
 [   0    0    0 ... 1385 2219 1907]
 [   0    0    0 ... 4080 4902 3639]
 ...
 [   0    0    0 ... 4612 2830  730]
 [   0    0    0 ... 1319  668 3047]
 [   0    0    0 ... 3898 3162 4268]]


#Creating Model for Train Set

In [13]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

In [16]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
200/200 [==============================] - 5s 9ms/step - loss: 0.3202 - accuracy: 0.8502 - val_loss: 0.1944 - val_accuracy: 0.9158
Epoch 2/10
200/200 [==============================] - 1s 7ms/step - loss: 0.1364 - accuracy: 0.9468 - val_loss: 0.2001 - val_accuracy: 0.9136
Epoch 3/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0950 - accuracy: 0.9662 - val_loss: 0.2289 - val_accuracy: 0.9154
Epoch 4/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0627 - accuracy: 0.9784 - val_loss: 0.3005 - val_accuracy: 0.9162
Epoch 5/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0385 - accuracy: 0.9885 - val_loss: 0.2899 - val_accuracy: 0.9116
Epoch 6/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0225 - accuracy: 0.9927 - val_loss: 0.3749 - val_accuracy: 0.9090
Epoch 7/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0130 - accuracy: 0.9967 - val_loss: 0.4784 - val_accuracy: 0.9127
Epoch 

###Performance Metrics And Accuracy



In [17]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

172/172 [==============================] - 1s 2ms/step


array([[2799,  308],
       [ 175, 2204]])

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9119577105359096